# Homework 4

In [23]:
import networkx as nx

## Visualize data

We chose the Google web graph at https://snap.stanford.edu/data/web-Google.html.

In [24]:
folder = 'data'
file = 'web-NotreDame.txt'

def read_graph(folder, file):
    with open(folder + '/' + file, 'r') as f:
        graph = nx.DiGraph()
        # skip first 4 lines
        for i in range(4):
            f.readline()
        # each line is an edge (pair of nodes)
        # counter = 0
        for line in f:
            # if counter == 1000:
            #     break
            edge = line.split()
            graph.add_edge(int(edge[0]), int(edge[1]))
            # counter += 1
    return graph

In [25]:
G = read_graph(folder, file)

print("Number of nodes: ", G.number_of_nodes())
print("Number of edges: ", G.number_of_edges())

Number of nodes:  325729
Number of edges:  1497134


In [ ]:
# visualize the graph
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.figure(figsize=(10, 10))
# nx.draw(G, node_size=10)
# plt.show()

In [26]:
# convert G to undirected graph
G_undirected = G.to_undirected()

print("Number of nodes: ", G_undirected.number_of_nodes())
print("Number of edges: ", G_undirected.number_of_edges())

Number of nodes:  325729
Number of edges:  1117563


In [28]:
# calculate number of triangles
triangles = nx.triangles(G_undirected)

print("Number of triangles: ", sum(triangles.values()) / 3)

Number of triangles:  8910005.0


## Sampling algorithm implementation

## Streaming graph algorithm

## Final questions

1. What were the challenges you faced when implementing the algorithm?
2. Can the algorithm be easily parallelized? If yes, how? If not, why? Explain.
3. Does the algorithm work for unbounded graph streams? Explain.
4. Does the algorithm support edge deletions? If not, what modification would it need? Explain.